# Madrid Air Quality 2018

The csv shows the air quality parameters (chemicals) measured at each of the stations in Madrid every 24 hours over the course of 4 months (January to April) in 2018. 

date BEN CH4 CO	EBE	NMHC NO	NO_2 NOx O_3 PM10 PM25 SO_2 TCH	TOL	station

However a lot of data is missing per station. And there is no one air quality feature to get from regression. Not sure if methods discussed in class would be appropriate to estimate a bunch of different chemicals.

In [1]:
import csv
import numpy as np
import numpy.linalg as la

with open('madrid_2018.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

# creating all data for station 28079004
data = np.array(data)
print(data.shape)
data = data[1:data.shape[0],0:data.shape[1]]
#print(data[0,:])
airq = []
date = []
for i in range(0, data.shape[0]):
    for j in range(0, data.shape[1]):
        if(data[i,-1]=='28079004'):
            if(j==0):
                date.append(data[i,j])
            elif(data[i,j]==''):
                airq.append('0')
            else:
                airq.append(data[i,j])
            
airq = np.array(airq)
date = np.array(date)
airq = np.reshape(airq, (2880,15))
print(airq.shape)
print(date.shape)
#print(date[2])
#print(airq[2,:])

airq = airq.astype(np.float)
#print(airq[3,:])

(69097, 16)
(2880, 15)
(2880,)


# Forecasting Temperature

In [2]:
with open('weatherHistory.csv', newline='') as csvfile:
    predata = list(csv.reader(csvfile))

# creating all data for station 28079004
predata = np.array(predata)
print(data.shape)
predata = predata[1:predata.shape[0],3:-1]
print(predata[0,:])
target = predata[:,0].astype(float)
data = predata[:,1:predata.shape[1]].astype(float)
print(target[0])
print(data[0,:])
print(target.shape)
print(data.shape)

(69096, 16)
['9.472222222222221' '7.3888888888888875' '0.89' '14.1197' '251.0'
 '15.826300000000002' '0.0' '1015.13']
9.472222222222221
[7.38888889e+00 8.90000000e-01 1.41197000e+01 2.51000000e+02
 1.58263000e+01 0.00000000e+00 1.01513000e+03]
(96453,)
(96453, 7)


In [3]:
def LeastSquares_RR(train,train_target,test,l):
    X = train
    #print(X.shape)
    t = train_target
    I = np.identity(X.shape[1])
    #t = train_target.reshape(len(train_target),1)
    #print(X.shape)
    what = la.inv(X.transpose().dot(X)+l*I).dot(X.transpose()).dot(t)
    y_hat = X.dot(what)
    lserror = la.norm(t - y_hat)**2/X.shape[0]
    
    y_pred = test.dot(what)
    return what, lserror, y_pred

X = data
t = target.reshape(96453,1)
ones = np.ones((96453,1))
X = np.column_stack((data,ones))
print(X.shape)
what, lserror, y_pred = LeastSquares_RR(X, t, X, 1)
print(lserror)

(96453, 8)
0.9005999860504107


In [6]:
def KFCV_RR(l, p):
           
    test1 = X[0:19290]; train1 = X[19290:38580]; 
    test1_t = t[0:19290]; train1_t = t[19290:96450]

    test2 = X[19290:38580]; train2 = np.vstack((X[0:19290],X[38580:96450])) 
    test2_t = t[19290:38580]; train2_t = np.vstack((t[0:19290],t[38580:96450]))

    test3 = X[38580:57870]; train3 = np.vstack((X[0:38580],X[57870:96450])) ; 
    test3_t = t[38580:57870]; train3_t = np.vstack((t[0:38580],t[57870:96450])) ; 

    test4 = X[57870:77160]; train4 = np.vstack((X[0:57870],X[77160:96450])); 
    test4_t = t[57870:77160]; train4_t = np.vstack((t[0:57870],t[77160:96450]));

    test5 = X[77160:96450]; train5 = X[0:77160]; 
    test5_t = t[77160:96450]; train5_t = t[0:77160]
    
    test = np.array([test1,test2,test3,test4,test5])
    test_t = np.array([test1_t,test2_t,test3_t,test4_t,test5_t])
    
    Errors = []
    for i in range(5):
        val = test[i]
        val_t = test_t[i]
        """ktt = np.asarray([[],[],[],[],[],[],[],[],[],[],[],[],[],[]]).transpose()
        ktt_t = np.asarray([[]]).transpose()
        for g in range(5):
            if g!=i:
                ktt = np.vstack((ktt,test[g]))
                ktt_t = np.vstack((ktt_t,test_t[g]))
        print(ktt)
        """
        ktt = []
        ktt_t = []
        for g in range(5):
            if g!=i:
                ktt.append(test[g])
                ktt_t.append(test_t[g])
        #print (test[:i].shape)
        #print (test[i+1].shape)
        """print (test.shape)
        ktt = np.vstack((test[:i],test[i+1:]))
        ktt_t = np.vstack((test_t[:i],test_t[i+1:]))
        """
        #print("len ktt is : ",len(ktt))
        whats = []
        es = []
        ypreds = []
        errors = []
        for j in range(4):
            testk = ktt[j]
            testk_t = ktt_t[j]
            train = []; train_t = []
            if len(ktt[:j])<=0:
                #print("----")
                train = np.vstack(ktt[j+1:])
                train_t = np.vstack(ktt_t[j+1:])
            elif ((j+1>=4) or len(ktt[j+1:])<0):
                #print("----ef")
                train = np.vstack(ktt[:j])
                train_t = np.vstack(ktt_t[:j])
            else:
                train = np.vstack(ktt[:j]+(ktt[j+1:]))
                train_t = np.vstack(ktt_t[:j]+ktt_t[j+1:])
                """print("----eee")
                if len(ktt[:j])==1:
                    train = ktt[:j]
                    train_t = ktt_t[:j]
                else:
                    train = np.vstack(ktt[:j])
                    train_t = np.vstack(ktt_t[:j])
                if len(ktt[j+1:])==1:
                    t2 = ktt[j+1:]
                    t2_t = ktt_t[j+1:]
                    train = np.vstack(train.append(t2))
                    train_t = np.vstack(train_t.append(t2))
                else:
                    t2 = np.vstack(ktt[j+1:])
                    t2_t = np.vstack(ktt_t[j+1:])
                
                    #print(train.shape,t2.shape)
                    #train = np.vstack((train,t2))
                    #train_t = np.vstack((train_t,t2_t))
                    train = np.vstack(train.append(t2))
                    train_t = np.vstack(train_t.append(t2))"""
            
            #print(train.shape)
            wh, e, ypred = LeastSquares_RR(train,train_t,testk,l)
            whats.append(wh)
            es.append(e)
            ypreds.append(ypred)
            error = la.norm(ypred - testk_t)**2/testk_t.shape[0]
            errors.append(error)
        kavg_error = np.mean(errors)
        idx = np.argmin(errors)
        yp = val.dot(whats[idx])
        new_e = la.norm(yp - val_t)**2/val_t.shape[0]
        Errors.append(new_e)
    return np.mean(Errors)

In [8]:
L = np.arange(0,3,0.001)
Errors = []
for l in L:
    Errors.append(KFCV_RR(l,False))
Errors = np.asarray(Errors)
i = np.argmin(Errors)
print("Optimal lambda is: ",L[i], "\nwith average prediction error: ",Errors[i])

LinAlgError: Singular matrix